In [76]:
import torch
import torch.nn as nn
from torch.nn.utils import weight_norm


class Chomp1d(nn.Module):
    def __init__(self, chomp_size):
        super(Chomp1d, self).__init__()
        self.chomp_size = chomp_size

    def forward(self, x):
        return x[:, :, :-self.chomp_size].contiguous()


class TemporalBlock(nn.Module):
    def __init__(self, n_inputs, n_outputs, kernel_size, stride, dilation, padding, ifrelu, dropout):
        super(TemporalBlock, self).__init__()
        self.conv1 = weight_norm(nn.Conv1d(n_inputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp1 = Chomp1d(padding)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)

        self.conv2 = weight_norm(nn.Conv1d(n_outputs, n_outputs, kernel_size,
                                           stride=stride, padding=padding, dilation=dilation))
        self.chomp2 = Chomp1d(padding)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout)

        self.net = nn.Sequential(self.conv1, self.chomp1, self.relu1, self.dropout1,
                                 self.conv2, self.chomp2, self.relu2, self.dropout2)
        self.downsample = nn.Conv1d(n_inputs, n_outputs, 1) if n_inputs != n_outputs else None
        self.ifrelu = ifrelu
        self.relu = nn.ReLU()
        self.init_weights()

    def init_weights(self):
        self.conv1.weight.data.normal_(0, 0.1)
        self.conv2.weight.data.normal_(0, 0.1)
        if self.downsample is not None:
            self.downsample.weight.data.normal_(0, 0.1)

    def forward(self, x):
        out = self.net(x)
        res = x if self.downsample is None else self.downsample(x)
        return out+res


class TemporalConvNet(nn.Module):
    def __init__(self, num_inputs, num_channels, kernel_size=4, dropout=0.1):
        super(TemporalConvNet, self).__init__()
        layers = []
        num_levels = len(num_channels)
        for i in range(num_levels):
            dilation_size = 2 ** i
            in_channels = num_inputs if i == 0 else num_channels[i-1]
            out_channels = num_channels[i]
            ifrelu = False if i==(len(num_channels)-1) else True
            layers += [TemporalBlock(in_channels, out_channels, kernel_size, stride=1, dilation=dilation_size,
                                     padding=(kernel_size-1) * dilation_size, ifrelu=ifrelu, dropout=dropout)]

        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return self.network(x)

In [65]:
import pandas as pd 
import numpy as np

result = pd.read_csv('drive/MyDrive/1_before2016.csv')
data = result.drop('meanest', axis=1)
x = data.drop('actual', axis=1)
mean_normal = np.mean(x, axis=0)
std_normal = np.std(x, axis=0)
data.loc[:, 'actq':'fincfy'] = (x - mean_normal)/std_normal

xseq = []
yseq = []
for name, content in data.groupby('tic'):
    for i in range(len(content.index)-10):
        train_seq = []
        train_label = []
        for j in range(i, i + 10):
            train_seq.append(content.iloc[j]['actual'])
        for j in range(i+1, i + 11):
            train_label.append(content.iloc[j]['actual'])
        xseq.append(train_seq)
        yseq.append(train_label)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3430: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3571: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)


In [66]:
result1 = pd.read_csv('drive/MyDrive/1_after2016.csv')
data1 = result1.drop('meanest', axis=1)
x1 = data1.drop('actual', axis=1)
data1.loc[:, 'actq':'fincfy'] = (x1 - mean_normal)/std_normal

xseqt = []
yseqt = []
for name, content in data1.groupby('tic'):
    for i in range(len(content.index)-10):
        train_seq = []
        train_label = []
        for j in range(i, i + 10):
            train_seq.append(content.iloc[j]['actual'])
        for j in range(i+1, i + 11):
            train_label.append(content.iloc[j]['actual'])
        xseqt.append(train_seq)
        yseqt.append(train_label)

In [79]:
import random

model = TemporalConvNet(num_inputs=10, num_channels=[1, 16, 16, 10])

train = [(xseq[i], yseq[i]) for i in range(len(xseq))]
random.shuffle(train)

N = len(xseq)
train_N = (int(N/64)-20) * 64
valid_N = N - train_N

train_x = torch.FloatTensor([train[i][0] for i in range(train_N)])
train_x = train_x.reshape(train_x.shape[0], -1, 1)
train_y = torch.FloatTensor([train[i][1] for i in range(train_N)])
train_y = train_y.reshape(train_y.shape[0], -1, 1)

valid_x = torch.FloatTensor([train[i][0] for i in range(train_N, N)])
valid_x = valid_x.reshape(valid_x.shape[0], -1, 1)
valid_y = torch.FloatTensor([train[i][1] for i in range(train_N, N)])
valid_y = valid_y.reshape(valid_y.shape[0], -1, 1)

test_x = torch.FloatTensor(xseqt)
test_x = test_x.reshape(test_x.shape[0], -1, 1)
test_y = torch.FloatTensor(yseqt)
test_y = test_y.reshape(test_y.shape[0], -1, 1)

In [84]:
criteria = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

In [ ]:
minloss = 0.08
for epoch in range(1000):
    total_loss = 0
    for step in range(int(train_x.shape[0]/64)):
        # 生成数据
        src = train_x[step*64 : (step*64+64)]
        tgt = train_y[step*64 : (step*64+64)]
        # 清空梯度
        optimizer.zero_grad()
        # 进行transformer的计算
        out = model(src)
        
        #print(out.contiguous().view(-1, out.size()[-1]))
        #print(tgt_y.contiguous().view(-1, tgt_y.size()[-1]))
        loss = criteria(out[:, -1, :], tgt[:, -1, :])
        #print(loss)

        # 计算梯度
        loss.backward()
        # 更新参数
        optimizer.step()

        total_loss += loss

        # 每10次打印一下loss
    if epoch % 10 == 0:
      print("Epoch {}, train_loss: {}".format(epoch, total_loss))
      model.eval()
      with torch.no_grad():
            v_src = valid_x
            v_tgt = valid_y
            v_out = model(v_src)
            v_loss = criteria(v_out[:, -1, :], v_tgt[:, -1, :])
            if(v_loss < minloss):
              minloss = v_loss
              torch.save(model, 'drive/MyDrive/tcn.pth')
            print("Epoch {}, valid_loss: {}".format(epoch, v_loss))
        

In [73]:
def withinten(y_true, y_pred):
    sum1 = 0
    for i in range(len(y_pred)):
        if(y_pred[i] < 1.1*y_true[i] and y_pred[i] > 0.9*y_true[i]):
            sum1+=1
    return sum1/len(y_pred)

In [ ]:
from sklearn.metrics import r2_score

def test(model, test_x, test_y):
  model.eval()
  with torch.no_grad():
      v_src = test_x
      v_tgt_y = test_y
      v_out = model(v_src)
      v_loss = criteria(v_out[:, -1, :], v_tgt_y[:, -1, :]).item()
      r2 = r2_score(v_tgt_y[:, -1, :], v_out[:, -1, :])
      within10 = withinten(v_tgt_y[:, -1, :], v_out[:, -1, :])
  return v_loss, r2, within10


In [ ]:
loss, r2, within10 = test(model, test_x, test_y)
loss, r2, within10